In [1]:
import pandas as pd

# Configuración para que muestre todas las columnas al completo
pd.set_option('display.max_columns', None)

# Configuración para que no muestre la notación científica
pd.set_option('display.float_format', '{:,.0f}'.format)

import warnings
warnings.filterwarnings("ignore")

In [2]:

def apertura_exploracion(csv):

    """ Función para leer csv, convertir a df y hacer una primera exploración.
    Igualar a variable con el nombre que quieres dar a DataFrame"""
    
    try:
        # Convertir el csv a DataFrame
        df = pd.read_csv(f"../data/{csv}.csv")        

        # Muestro las primeras filas
        display(df.head())

        # Muestro las últimas filas
        display(df.tail())

        # Muestro las dimensiones del dataframe
        print(f"-----\n\nEl DataFrame tiene {df.shape[0]} filas y {df.shape[1]} columnas.\n-----")

        # Consulto si hay filas duplicadas
        print(f"\nEl número de filas duplicadas es {df.duplicated().sum()}\n-----")

        # Muestro el tipo de dato y si hay nulos por cada columna
        print("\nInformación del DataFrame:")
        df.info()

        # Muestro el porcentaje de nulos por variable
        display(round(df.isnull().mean() * 100, 2))


        # Muestro las estadísticas de columnas numéricas
        print("-----\n\nEstadísticas descriptivas:")
        display(df.describe(include="all").T)

        # Me devuelve un df que tendré que igualar a una variable
        return df  
                

    # Excepciones en caso de no encontrar el archivo o de que haya un error
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo '../data/{csv}.csv'.")
        return None  
    
    except Exception as e:
        print(f"Ocurrió un error: {e}")
        return None 

In [3]:
df_movies = apertura_exploracion("raw_movies_dataset")

,Title,Genre,Year,Budget,Revenue,IMDB_Rating
0,The Last Journey,Drama,2020,11000000,"34,049,694",5
1,Infinite Dreams,Drama,2004,178000000,"550,436,005",7
2,Code of Shadows,Romance,2018,13000000,"7,855,773",5
3,Ocean Whisper,Action,2016,148000000,"176,997,903",7
4,Forgotten Realm,Romance,2017,61000000,"126,406,131",NaN


,Title,Genre,Year,Budget,Revenue,IMDB_Rating
17,Shattered Skies,NaN,2008,59000000,"218,062,610",7
18,Eternal Voyage,Fantasy,2022,59000000,NaN,6
19,Mystic River,NaN,2020,41000000,"58,510,304",5
20,The Last Journey,Drama,2020,11000000,"34,049,694",5
21,Shattered Skies,NaN,2008,59000000,"218,062,610",7


-----

El DataFrame tiene 22 filas y 6 columnas.
-----

El número de filas duplicadas es 2
-----

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        22 non-null     object 
 1   Genre        18 non-null     object 
 2   Year         22 non-null     object 
 3   Budget       22 non-null     object 
 4   Revenue      19 non-null     float64
 5   IMDB_Rating  18 non-null     float64
dtypes: float64(2), object(4)
memory usage: 1.2+ KB


Title          0
Genre         18
Year           0
Budget         0
Revenue       14
IMDB_Rating   18
dtype: float64

-----

Estadísticas descriptivas:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Title,22,20,The Last Journey,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Genre,18,7,Drama,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Year,22,15,2020,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Budget,22,19,59000000,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Revenue,19,NaN,NaN,NaN,"173,447,916","136,688,547","7,855,773","59,656,930","150,236,576","221,459,089","550,436,005"
IMDB_Rating,18,NaN,NaN,NaN,6,1,4,5,6,7,9
